In [1]:
!pip install --quiet -U langgraph langchain_ollama langchain_core langchain_openai mermaid-python
import os
os.environ["OPENAI_API_KEY"] = os.environ.get("OPEN_AI_KEY")

# Goals
- We can extend a router, into a generic agent architecture. We return a `ToolMessage` to a user, when we call a tool.
- What happens if we pass a `ToolMessage` back to a model?

# ReAct
- **ReAct** is a general agent architecture:
    - `act` - let the model call specific tools
    - `observe` - pass the tool output back to the model
    - `reason` - let the model reason about the tool output to decide what to do next.